In [1]:
import psycopg2
import sqlalchemy
import sqlalchemy.orm
import os
import yaml

In [2]:
def connect_to_snapshot(yaml_path):
    file = open(yaml_path, "r")
    db_info = yaml.load(file)
    file.close()
    url = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(db_info["username"],
                                           db_info["password"],
                                           db_info["host"],
                                           db_info["port"],
                                           db_info["dbname"])
    engine = sqlalchemy.create_engine(url)
    conn = engine.connect()
    meta = sqlalchemy.MetaData(bind = engine)
    meta.reflect()
    return engine, conn, meta

In [3]:
engine, conn, meta = connect_to_snapshot("ml-snapshot.yml")

In [4]:
inspector = sqlalchemy.engine.reflection.Inspector.from_engine(engine)
inspector.get_columns("facebook_pages")

[{'autoincrement': False,
  'default': None,
  'name': 'id',
  'nullable': False,
  'type': BIGINT()},
 {'autoincrement': False,
  'default': None,
  'name': 'entity',
  'nullable': True,
  'type': JSONB(astext_type=Text())},
 {'autoincrement': False,
  'default': None,
  'name': 'created_at',
  'nullable': False,
  'type': TIMESTAMP()},
 {'autoincrement': False,
  'default': None,
  'name': 'updated_at',
  'nullable': False,
  'type': TIMESTAMP()},
 {'autoincrement': False,
  'default': None,
  'name': 'image',
  'nullable': True,
  'type': VARCHAR()},
 {'autoincrement': False,
  'default': None,
  'name': 'name',
  'nullable': True,
  'type': VARCHAR()}]

In [5]:
FacebookPages = meta.tables["facebook_pages"]

In [7]:
from sqlalchemy.sql import select, func
s = select([func.count(FacebookPages.c.id)])
conn.execute(s).fetchall()

[(157,)]

In [8]:
s = select([FacebookPages.c.entity])
sample_user = conn.execute(s).fetchone()[0]
loop = [print(key) for key in sorted(sample_user.keys())]

id
name


In [30]:
s = select([FacebookPages]).where(FacebookPages.c.name != None)
conn.execute(s).fetchone()

(166606563384885, {'id': '166606563384885', 'name': 'Garvin Thomas NBC Bay Area'}, datetime.datetime(2016, 12, 2, 18, 36, 41, 203703), datetime.datetime(2016, 12, 10, 9, 30, 25, 376622), 'https://scontent.xx.fbcdn.net/v/t1.0-1/p50x50/12140774_941429039235963_4992205500873324218_n.jpg?oh=354d98afc78670a3085509898a6e950b&oe=58BBE30D', 'Garvin Thomas NBC Bay Area')

In [43]:
import requests
file = open("api-keys.yml", "r")
api_keys = yaml.load(file)
file.close()
url_prefix = r"https://graph.facebook.com/v2.8/"
access_token = r"access_token={}".format(api_keys["facebook"]["access-token"])

In [44]:
requests.get(url_prefix + r"166606563384885?" + access_token).text

'{"name":"Garvin Thomas NBC Bay Area","id":"166606563384885"}'

In [52]:
r = eval(requests.get(url_prefix + r"166606563384885/likes?" + access_token).text)
r

{'data': [{'id': '66731150860', 'name': 'Compassion Without Borders'},
  {'id': '466896256668273', 'name': 'Julie Kramer'},
  {'id': '802845313141599', 'name': 'Phil Lipof'},
  {'id': '124334300676', 'name': 'Valley Medical Center Foundation'},
  {'id': '1079485892075077', 'name': 'Peggy Bunker'},
  {'id': '427583733952709', 'name': 'Laura Garcia Cannon'},
  {'id': '189383104414761', 'name': 'Stephanie Chuang'},
  {'id': '194851267219866', 'name': 'Hunger at Home'},
  {'id': '909584059137981', 'name': 'Cranksgiving San Jose'},
  {'id': '27104741639', 'name': 'Second Harvest Food Bank'},
  {'id': '463463187187304', 'name': 'Zoë Bands'},
  {'id': '1655319388064095', 'name': 'Project Open Paw'},
  {'id': '73282863973', 'name': 'Dave Huddleston'},
  {'id': '1447032725535735', 'name': 'Ambassadors of Hope'},
  {'id': '1574490369484680', 'name': 'Melody Gonzales'},
  {'id': '561973903841519', 'name': 'South Bay Clean Creeks Coalition'},
  {'id': '310820892395037', 'name': 'SnickerdoodALZ'},


In [57]:
r = eval(requests.get(url_prefix + r"166606563384885/likes?" + access_token + "&after=" + r["paging"]["cursors"]["after"]).text)
r

{'data': [{'id': '55375096989', 'name': 'NBC Bay Area'},
  {'id': '119314614790209', 'name': 'Cory Marshall'}],
 'paging': {'cursors': {'after': 'MTE5MzE0NjE0NzkwMjA5',
   'before': 'NTUzNzUwOTY5ODkZD'},
  'previous': 'https:\\/\\/graph.facebook.com\\/v2.8\\/166606563384885\\/likes?access_token=369339830067115%7C-gqJBkWfEhsEuMPopn-mh6gJml8&limit=25&before=NTUzNzUwOTY5ODkZD'}}